In [1]:
import numpy as np
from tensorflow.keras.models import load_model
import tensorflow as tf
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'

2024-06-08 02:03:36.118516: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-06-08 02:03:36.229400: I tensorflow/core/util/port.cc:104] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.


In [8]:
def end1(board):
    # 보드의 가로, 세로, 대각선을 모두 확인하여 종료 조건을 확인
    for i in range(19):
        for j in range(19):
            # 가로 확인
            if j + 5 <= 19:
                if np.all(board[i, j:j+5] == 1):
                    print('END')
                    return True
            # 세로 확인
            if i + 5 <= 19:
                if np.all(board[i:i+5, j] == 1):
                    print('END')
                    return True
            # 대각선 확인 (오른쪽 아래 방향)
            if i + 5 <= 19 and j + 5 <= 19:
                if np.all(np.diag(board[i:i+5, j:j+5]) == 1):
                    print('END')
                    return True
            # 대각선 확인 (왼쪽 아래 방향)
            if i + 5 <= 19 and j - 5 >= -1:
                if np.all(np.diag(np.fliplr(board[i:i+5, j-4:j+1])) == 1):
                    print('END')
                    return True
    return False

def end2(board):
    # 보드의 가로, 세로, 대각선을 모두 확인하여 종료 조건을 확인
    for i in range(19):
        for j in range(19):
            # 가로 확인
            if j + 5 <= 19:
                if np.all(board[i, j:j+5] == -1):
                    print('END')
                    return True
            # 세로 확인
            if i + 5 <= 19:
                if np.all(board[i:i+5, j] == -1):
                    print('END')
                    return True
            # 대각선 확인 (오른쪽 아래 방향)
            if i + 5 <= 19 and j + 5 <= 19:
                if np.all(np.diag(board[i:i+5, j:j+5]) == -1):
                    print('END')
                    return True
            # 대각선 확인 (왼쪽 아래 방향)
            if i + 5 <= 19 and j - 5 >= -1:
                if np.all(np.diag(np.fliplr(board[i:i+5, j-4:j+1])) == -1):
                    print('END')
                    return True
    return False


def print_board(board):
    return
    for i in board:
        print(i)

In [9]:
# with tf.device('/CPU:0'):
    print('=' * 50)
    print()
    model = load_model('/home/jinjinjara1022/OmokAI/models/best.h5')
    board = np.zeros((19, 19), dtype=np.int8)
    first_turn, location_r, location_c = list(input('선공인가요?(Y/N) 첫번째 착수 위치를 알려주세요.').split())
    print()
    print('=' * 50)
    if first_turn == 'Y':
        board[19 - int(location_r), ord(location_c)-65] = 1
    else:
        board[19 - int(location_r), ord(location_c)-65] = -1
    
    print_board(board)
    
    if first_turn != 'Y':
        input_ = board.copy()
        input_ = np.expand_dims(input_, axis=(0, -1)).astype(np.float32)
        output = model.predict(input_, verbose=0).squeeze()
        output = output.reshape((19, 19))
        output_y, output_x = np.unravel_index(np.argmax(output), output.shape)
        if board[output_y, output_x] == 0:
            board[output_y, output_x] = 1
        else:
            while True:
                output[output_y, output_x] = -np.inf
                output_y, output_x = np.unravel_index(np.argmax(output), output.shape)
                if board[output_y, output_x] == 0:
                    board[output_y, output_x] = 1
                    break
        print_board(board)
        print()
        print(f"착수한 위치 : {19-output_y}행 {chr(output_x + 65)}열")
        print()
        print('=' * 50)
    
    while True:
        if end1(board):
            print('P1 승리')
            break
        if end2(board):
            print('P2 승리')
            break
            
        print()
        location_r, location_c = list(input("상대방이 착수 한 곳을 입력해 주세요").split())
        if board[19-int(location_r), ord(location_c)-65] != 0:
            print("해당 위치는 이미 착수 되었습니다.")
            continue
        board[19-int(location_r), ord(location_c)-65] = -1

        
                
        
        input_ = board.copy()
        input_ = np.expand_dims(input_, axis=(0, -1)).astype(np.float32)
        output = model.predict(input_, verbose=0).squeeze()
        output = output.reshape((19, 19))
        output_y, output_x = np.unravel_index(np.argmax(output), output.shape)
        if board[output_y, output_x] == 0:
            board[output_y, output_x] = 1
        else:
            while True:
                output[output_y, output_x] = -np.inf
                output_y, output_x = np.unravel_index(np.argmax(output), output.shape)
                if board[output_y, output_x] == 0:
                    board[output_y, output_x] = 1
                    break
        print()
        print('=' * 50)
        print()
        print(f"내가 착수한 위치 : {19-output_y}행 {chr(output_x + 65)}열")
        print()
        print('=' * 50)
        print_board(board)

선공인가요?(Y/N) 첫번째 착수 위치를 알려주세요. N 10 J




착수한 위치 : 8행 I열




상대방이 착수 한 곳을 입력해 주세요 10 K




내가 착수한 위치 : 10행 I열




상대방이 착수 한 곳을 입력해 주세요 9 I


...



내가 착수한 위치 : 18행 F열




상대방이 착수 한 곳을 입력해 주세요 16 K




내가 착수한 위치 : 16행 O열




상대방이 착수 한 곳을 입력해 주세요 16 J




내가 착수한 위치 : 16행 H열

END
P2 승리
